# ResNet

    '''
    参数设置参考Justin 598WI2022课件
    
    ResNet-18

    5轮训练: 11m 20.6s;
    100%|██████████| 782/782 [02:16<00:00,  5.73it/s]
    Epoch [2/5], Loss: 0.9377
    Epoch [3/5], Loss: 0.7488
    Epoch [4/5], Loss: 0.6400
    Epoch [5/5], Loss: 0.5503

    Test Accuracy: 79.54%

    ----------------------------
    ResNet-34

    5轮训练: 9m 11.5s; （速度有些异常，可能用的是bottleneck Block，速度很快，效果一般）
    100%|██████████| 782/782 [01:48<00:00,  7.22it/s]
    Epoch [2/5], Loss: 0.8834
    Epoch [3/5], Loss: 0.6894
    Epoch [4/5], Loss: 0.5648
    Epoch [5/5], Loss: 0.4769
    Test Accuracy: 76.89%

    5轮训练: 16m 3.9s; (比较合理的结果)
    100%|██████████| 782/782 [03:20<00:00,  3.89it/s]  
    Epoch [1/5], Loss: 1.4514
    Epoch [2/5], Loss: 0.9806
    Epoch [3/5], Loss: 0.7942
    Epoch [4/5], Loss: 0.6848
    Epoch [5/5], Loss: 0.6016
    Test Accuracy: 79.06%

    ----------------------------
    ResNet-50 
    
    5轮训练: 26m 17.5s;
    100%|██████████| 782/782 [05:20<00:00,  2.44it/s]
    Epoch [1/5], Loss: 1.7421
    Epoch [2/5], Loss: 1.4417
    Epoch [3/5], Loss: 1.3398
    Epoch [4/5], Loss: 1.2915
    Epoch [5/5], Loss: 1.2611
    Test Accuracy: 46.44% （Q:有问题，训练不起来, A: Block里没加ReLU, 玄学）

    5轮训练: 26m 6.8s;
    100%|██████████| 782/782 [05:16<00:00,  2.47it/s]
    Epoch [2/5], Loss: 0.8592
    Epoch [3/5], Loss: 0.6755
    Epoch [4/5], Loss: 0.5729
    Epoch [5/5], Loss: 0.4987
    Test Accuracy: 77.41%

    improving Residual Net (修改relu顺序)
    5轮训练: 27m 23.0s;
    100%|██████████| 782/782 [05:30<00:00,  2.37it/s]
    Epoch [1/5], Loss: 1.4292
    Epoch [2/5], Loss: 0.9679
    Epoch [3/5], Loss: 0.7916
    Epoch [4/5], Loss: 0.6479
    Epoch [5/5], Loss: 0.5383
    Test Accuracy: 79.27% （improving 方法是好一点，但也可能是训练随机性）

    官方实现 
    5轮训练: 27m 23.9s;
    100%|██████████| 782/782 [05:27<00:00,  2.39it/s]
    Epoch [2/5], Loss: 1.0033
    Epoch [3/5], Loss: 0.7206
    Epoch [4/5], Loss: 0.5603
    Epoch [5/5], Loss: 0.4549
    Test Accuracy: 78.62%
    '''

## Basic Block
<img src="./image/resnet/resnet1.png" alt="Model Image" width="800">
<img src="./image/resnet/resnet2.png" alt="Model Image" width="800">
<img src="./image/resnet/resnet3.png" alt="Model Image" width="800">

## ResNet-18 | ResNet-34
<img src="./image/resnet/resnet4.png" alt="Model Image" width="800">

## Bottleneck Block
<img src="./image/resnet/resnet5.png" alt="Model Image" width="800">
<img src="./image/resnet/resnet6.png" alt="Model Image" width="800">

## Improving
<img src="./image/resnet/resnet7.png" alt="Model Image" width="800">



In [ ]:
# 环境配置
%cd ../../
import sys
sys.path.append('./python')

In [ ]:
from tqdm import tqdm
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader

import torchvision
from torchvision import transforms

from sgd_cv.model import ResNet18, ResNet34, ResNet50, ResNet101, ResNet152

import torchvision.models as models

## 测试模型

In [ ]:
# 测试模型结构
model = ResNet50(num_classes=10)
# print(model)

# 随机生成一个批次的输入 (cifar10 图像大小: 3x227x227)
input_tensor = torch.randn(1, 3, 224, 224) # 是否设为224
output = model(input_tensor)

print(f"输入张量大小: {input_tensor.shape}")
print(f"输出张量大小: {output.shape}")  # 应为 [1, 10]


## 训练模型

In [ ]:
# 数据集加载
transform = transforms.Compose([transforms.Resize(256),
                                transforms.CenterCrop(224), # 224
                                transforms.ToTensor(),
                                transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                     std=[0.229, 0.224, 0.225]),
                                ])
# CIFAR10
train_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.CIFAR10('./data/CIFAR10/', train=False, transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# 定义模型、损失函数和优化器
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# model = models.resnet50().to(device) # 对比一下官方模型效果
model = ResNet50(num_classes=10).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=3e-4)
# 启用异常检测
torch.autograd.set_detect_anomaly(False)

In [ ]:
# 训练
num_epochs = 5
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)

        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播和优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

In [ ]:
# torch.save(model, 'model.pth')
# model = torch.load('model.pth')

In [ ]:
# 测试
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")